In [72]:
!pip install geopy

You should consider upgrading via the 'C:\Users\leano\PycharmProjects\clusteringRestaurants\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [73]:
# Clustering for Vancouver
import pandas as pd
import re
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter


# load total population of each Vancouver neighbourhood
income_path = pd.read_csv('income_Vancouver1.csv', index_col= 'Total income')
df_income = pd.DataFrame(income_path)
df_income.loc['Total',:]= df_income.sum(axis=0)
df_income

,Arbutus-Ridge,Downtown,Dunbar-Southlands,Fairview,Grandview-Woodland,Hastings-Sunrise,Kensington-Cedar Cottage,Kerrisdale,Killarney,Kitsilano,...,Riley Park,Shaughnessy,South Cambie,Strathcona,Sunset,Victoria-Fraserview,West End,West Point Grey,Vancouver CSD,Vancouver CMA
Total income,,,,,,,,,,,,,,,,,,,,,
Under $10.000 (including loss),2790.0,8785.0,3320.0,3185.0,3300.0,4575.0,6595.0,2285.0,4160.0,4960.0,...,2705.0,1190.0,920.0,1530.0,5155.0,4840.0,5920.0,2020.0,86590.0,338730.0
$10.000 to $19.999,1820.0,8050.0,2140.0,3495.0,5305.0,6315.0,8580.0,1650.0,4700.0,4945.0,...,2860.0,665.0,815.0,3430.0,6800.0,5720.0,7225.0,1370.0,95255.0,346355.0
$20.000 to $29.999,1365.0,4830.0,1615.0,3110.0,3420.0,4390.0,5885.0,1295.0,3165.0,3875.0,...,2260.0,650.0,750.0,1080.0,4585.0,3940.0,5685.0,985.0,66640.0,248170.0
$30.000 to $39.999,1110.0,4325.0,1205.0,2955.0,2945.0,3185.0,4820.0,865.0,2570.0,3435.0,...,1870.0,475.0,565.0,575.0,3600.0,3085.0,4925.0,845.0,54720.0,210570.0
$40.000 to $49.999,860.0,4270.0,1130.0,3030.0,2590.0,2580.0,3865.0,795.0,2225.0,3510.0,...,1760.0,490.0,495.0,435.0,2850.0,2360.0,4860.0,785.0,48860.0,188140.0
$50.000 to $59.999,700.0,3700.0,1025.0,2630.0,1955.0,1895.0,2915.0,680.0,1735.0,3110.0,...,1310.0,380.0,450.0,385.0,1820.0,1615.0,3825.0,690.0,38355.0,149530.0
$60.000 to $69.999,620.0,3530.0,820.0,2370.0,1545.0,1465.0,1965.0,670.0,1320.0,2410.0,...,1075.0,350.0,400.0,200.0,1240.0,1065.0,2945.0,500.0,30150.0,115955.0
$70.000 to $79.999,470.0,3115.0,705.0,1980.0,1170.0,1155.0,1490.0,460.0,950.0,2160.0,...,930.0,270.0,385.0,210.0,780.0,780.0,2385.0,555.0,24120.0,89390.0
$80.000 to $89.999,415.0,2495.0,635.0,1625.0,850.0,730.0,1245.0,370.0,910.0,1805.0,...,785.0,285.0,315.0,210.0,670.0,640.0,1660.0,450.0,19175.0,71045.0


In [74]:
'''
cnt = 0
for column in df_income:
    for row in df_income[column]:
        total = df_income.loc["Total",column]
        value = (row/total)*100
        df_income.loc[cnt,column] = value
        cnt +=1
df_income
'''

'\ncnt = 0\nfor column in df_income:\n    for row in df_income[column]:\n        total = df_income.loc["Total",column]\n        value = (row/total)*100\n        df_income.loc[cnt,column] = value\n        cnt +=1\ndf_income\n'

In [75]:
# load citizenship dataset
citizen_path = pd.read_csv('citizenship_Vancouver1.csv'  )
df_citizen = pd.DataFrame(citizen_path)
df_citizen

,ID,Total - Selected places of birth for the immigrant population in private households,Arbutus-Ridge,Downtown,Dunbar-Southlands,Fairview,Grandview-Woodland,Hastings-Sunrise,Kensington-Cedar Cottage,Kerrisdale,...,Riley Park,Shaughnessy,South Cambie,Strathcona,Sunset,Victoria-Fraserview,West End,West Point Grey,Vancouver CSD,Vancouver CMA
0,2583,Americas,390.0,2445.0,785.0,1465.0,1050.0,1110.0,1285.0,305.0,...,585.0,195.0,220.0,340.0,935.0,665.0,2545.0,550.0,20600.0,65330.0
1,2584,Brazil,15.0,190.0,20.0,105.0,55.0,50.0,50.0,0.0,...,10.0,0.0,10.0,0.0,35.0,30.0,230.0,10.0,1030.0,2855.0
2,2585,Colombia,20.0,160.0,20.0,50.0,35.0,45.0,85.0,0.0,...,25.0,10.0,0.0,0.0,50.0,55.0,110.0,20.0,950.0,3080.0
3,2586,El Salvador,0.0,25.0,10.0,25.0,65.0,110.0,115.0,0.0,...,15.0,0.0,0.0,30.0,125.0,130.0,85.0,0.0,1125.0,4400.0
4,2587,Guyana,0.0,25.0,10.0,15.0,15.0,15.0,10.0,0.0,...,10.0,0.0,10.0,10.0,25.0,10.0,35.0,0.0,255.0,920.0
5,2588,Haiti,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,20.0,260.0
6,2589,Jamaica,25.0,35.0,25.0,45.0,0.0,20.0,25.0,15.0,...,10.0,0.0,10.0,0.0,40.0,30.0,65.0,10.0,485.0,2230.0
7,2590,Mexico,10.0,440.0,70.0,190.0,125.0,130.0,145.0,40.0,...,40.0,30.0,15.0,50.0,90.0,85.0,570.0,45.0,2760.0,7855.0
8,2591,Peru,25.0,85.0,20.0,50.0,20.0,65.0,75.0,0.0,...,30.0,20.0,0.0,10.0,25.0,35.0,55.0,35.0,790.0,2695.0
9,2592,Trinidad and Tobago,15.0,30.0,10.0,45.0,10.0,10.0,30.0,10.0,...,10.0,0.0,10.0,10.0,50.0,15.0,65.0,0.0,455.0,1940.0


In [76]:
# preprocessing
df_citizen.drop(['ID'], axis=1, inplace=True)
# get the transposed dataset
df_citizen = df_citizen.transpose()
df_citizen.columns = df_citizen.iloc[0]
#remove first row from DataFrame
df_citizen = df_citizen[1:]
df_citizen

Total - Selected places of birth for the immigrant population in private households,Americas,Brazil,Colombia,El Salvador,Guyana,Haiti,Jamaica,Mexico,Peru,Trinidad and Tobago,...,NaN,Lebanon,Pakistan,Philippines,Sri Lanka,Syria,Taiwan,Viet Nam,Other places of birth in Asia,Oceania and other places of birth
Arbutus-Ridge,390.0,15.0,20.0,0.0,0.0,0.0,25.0,10.0,25.0,15.0,...,NaN,15.0,10.0,105.0,10.0,20.0,620.0,30.0,235.0,85.0
Downtown,2445.0,190.0,160.0,25.0,25.0,0.0,35.0,440.0,85.0,30.0,...,NaN,40.0,80.0,795.0,55.0,40.0,910.0,230.0,1310.0,360.0
Dunbar-Southlands,785.0,20.0,20.0,10.0,10.0,0.0,25.0,70.0,20.0,10.0,...,NaN,10.0,10.0,170.0,25.0,0.0,580.0,20.0,290.0,155.0
Fairview,1465.0,105.0,50.0,25.0,15.0,0.0,45.0,190.0,50.0,45.0,...,NaN,25.0,35.0,450.0,10.0,10.0,420.0,45.0,420.0,275.0
Grandview-Woodland,1050.0,55.0,35.0,65.0,15.0,0.0,0.0,125.0,20.0,10.0,...,NaN,15.0,25.0,590.0,15.0,10.0,70.0,515.0,205.0,235.0
Hastings-Sunrise,1110.0,50.0,45.0,110.0,15.0,0.0,20.0,130.0,65.0,10.0,...,NaN,0.0,15.0,1245.0,45.0,15.0,165.0,1540.0,590.0,250.0
Kensington-Cedar Cottage,1285.0,50.0,85.0,115.0,10.0,0.0,25.0,145.0,75.0,30.0,...,NaN,10.0,50.0,4385.0,225.0,20.0,225.0,2585.0,710.0,475.0
Kerrisdale,305.0,0.0,0.0,0.0,0.0,0.0,15.0,40.0,0.0,10.0,...,NaN,25.0,10.0,235.0,0.0,0.0,615.0,40.0,180.0,75.0
Killarney,855.0,35.0,70.0,90.0,25.0,0.0,0.0,85.0,65.0,35.0,...,NaN,0.0,55.0,2015.0,60.0,0.0,535.0,770.0,860.0,215.0
Kitsilano,1905.0,60.0,70.0,35.0,15.0,0.0,40.0,190.0,50.0,25.0,...,NaN,30.0,40.0,250.0,35.0,20.0,300.0,95.0,485.0,415.0


In [77]:
# put together places of birth of South-east- Asians, Japanese, Europeans, Western, ..
df_citizen['Total Population'] = df_citizen.iloc[:,0]+df_citizen.iloc[:,12]+df_citizen.iloc[:,30]+df_citizen.iloc[:,40]

df_citizen['Percentage of Latin-Americans'] = (df_citizen.iloc[:,1:9].sum(axis=1)+df_citizen.iloc[:,11] )/ df_citizen['Total Population'] * 100
df_citizen['Percentage of Europeans'] = (df_citizen.iloc[:,12] / df_citizen['Total Population']) * 100
df_citizen['Percentage of US-Americans'] = (df_citizen.iloc[:,10] / df_citizen['Total Population']) * 100
df_citizen['Percentage of Asians'] = (df_citizen.iloc[:,40] / df_citizen['Total Population']) * 100
df_citizen['Percentage of Japanese'] = (df_citizen.iloc[:,48]/ df_citizen['Total Population']) * 100
df_citizen['Percentage of Chinese'] = (df_citizen.iloc[:,43]/ df_citizen['Total Population']) * 100
df_citizen['Percentage of South-East-Asians'] = ((df_citizen.iloc[:,56]+df_citizen.iloc[:,52]) / df_citizen['Total Population']) * 100

df_citizen.drop(df_citizen.iloc[:,0:59], axis=1, inplace=True)
df_citizen.head()

Total - Selected places of birth for the immigrant population in private households,Total Population,Percentage of Latin-Americans,Percentage of Europeans,Percentage of US-Americans,Percentage of Asians,Percentage of Japanese,Percentage of Chinese,Percentage of South-East-Asians
Arbutus-Ridge,7650.0,1.699346,14.117647,3.137255,78.69281,1.24183,43.464052,1.764706
Downtown,23240.0,5.658348,24.010327,4.668675,61.574871,2.560241,14.350258,4.410499
Dunbar-Southlands,7810.0,3.008963,20.99872,6.850192,66.453265,1.024328,37.900128,2.432778
Fairview,9295.0,6.777838,35.233997,8.552985,43.410436,2.582033,10.220549,5.325444
Grandview-Woodland,7480.0,6.684492,29.745989,7.219251,53.208556,1.938503,22.526738,14.772727


In [78]:
df_citizen = df_citizen.transpose()
df_citizen

,Arbutus-Ridge,Downtown,Dunbar-Southlands,Fairview,Grandview-Woodland,Hastings-Sunrise,Kensington-Cedar Cottage,Kerrisdale,Killarney,Kitsilano,...,Riley Park,Shaughnessy,South Cambie,Strathcona,Sunset,Victoria-Fraserview,West End,West Point Grey,Vancouver CSD,Vancouver CMA
Total - Selected places of birth for the immigrant population in private households,,,,,,,,,,,,,,,,,,,,,
Total Population,7650.0,23240.0,7810.0,9295.0,7480.0,15050.0,22135.0,6560.0,14900.0,11160.0,...,7430.0,3490.0,2565.0,3825.0,19640.0,17800.0,15585.0,4665.0,257475.0,965665.0
Percentage of Latin-Americans,1.699346,5.658348,3.008963,6.777838,6.684492,4.883721,3.54642,1.295732,4.060403,6.317204,...,3.364738,2.43553,3.508772,3.921569,3.258656,3.061798,9.496311,3.644159,4.212059,3.825343
Percentage of Europeans,14.117647,24.010327,20.99872,35.233997,29.745989,15.415282,10.61667,11.204268,8.758389,43.996416,...,19.380888,13.753582,19.298246,11.895425,4.913442,5.926966,37.856914,25.937835,17.085154,17.700755
Percentage of US-Americans,3.137255,4.668675,6.850192,8.552985,7.219251,2.358804,2.100745,2.972561,1.409396,10.528674,...,4.643338,2.86533,4.678363,4.575163,1.272912,0.589888,6.352262,8.252947,3.610059,2.739045
Percentage of Asians,78.69281,61.574871,66.453265,43.410436,53.208556,75.913621,82.042015,82.469512,82.818792,35.080645,...,70.592194,76.934097,68.810916,73.986928,89.231161,89.297753,41.931344,59.163987,72.515778,72.209824
Percentage of Japanese,1.24183,2.560241,1.024328,2.582033,1.938503,0.830565,1.106844,1.448171,1.14094,2.553763,...,2.220727,0.859599,2.923977,1.045752,0.534623,0.589888,4.138595,2.250804,1.629284,1.105974
Percentage of Chinese,43.464052,14.350258,37.900128,10.220549,22.526738,37.674419,31.420827,41.768293,27.751678,8.915771,...,20.861373,41.404011,18.71345,44.705882,14.180244,37.724719,4.940648,33.118971,25.926789,19.56838
Percentage of South-East-Asians,1.764706,4.410499,2.432778,5.325444,14.772727,18.504983,31.488593,4.192073,18.691275,3.091398,...,21.601615,3.295129,8.382066,12.941176,26.909369,21.179775,5.229387,1.178992,15.69667,12.500194


In [79]:
df_new = pd.concat([df_citizen,df_income])
df_new = df_new.transpose()
lst = list(df_new.index)
for element in lst:
    df_new.loc[element, 'Neighbourhood'] = element + ',Vancouver,BC,Canada'
df_new = df_new.drop(df_new.index[23])
df_new = df_new.drop(df_new.index[22])
df_new

,Total Population,Percentage of Latin-Americans,Percentage of Europeans,Percentage of US-Americans,Percentage of Asians,Percentage of Japanese,Percentage of Chinese,Percentage of South-East-Asians,Under $10.000 (including loss),$10.000 to $19.999,...,$50.000 to $59.999,$60.000 to $69.999,$70.000 to $79.999,$80.000 to $89.999,$90.000 to $99.999,$100.000 and over,$100.000 to $149.999,$150.000 and over,Total,Neighbourhood
Arbutus-Ridge,7650.0,1.699346,14.117647,3.137255,78.69281,1.24183,43.464052,1.764706,2790.0,1820.0,...,700.0,620.0,470.0,415.0,320.0,1720.0,855.0,870.0,13915.0,"Arbutus-Ridge ,Vancouver,BC,Canada"
Downtown,23240.0,5.658348,24.010327,4.668675,61.574871,2.560241,14.350258,4.410499,8785.0,8050.0,...,3700.0,3530.0,3115.0,2495.0,1940.0,8590.0,5180.0,3405.0,62215.0,"Downtown ,Vancouver,BC,Canada"
Dunbar-Southlands,7810.0,3.008963,20.99872,6.850192,66.453265,1.024328,37.900128,2.432778,3320.0,2140.0,...,1025.0,820.0,705.0,635.0,575.0,3515.0,1555.0,1965.0,20205.0,"Dunbar-Southlands ,Vancouver,BC,Canada"
Fairview,9295.0,6.777838,35.233997,8.552985,43.410436,2.582033,10.220549,5.325444,3185.0,3495.0,...,2630.0,2370.0,1980.0,1625.0,1165.0,4105.0,2590.0,1520.0,33760.0,"Fairview ,Vancouver,BC,Canada"
Grandview-Woodland,7480.0,6.684492,29.745989,7.219251,53.208556,1.938503,22.526738,14.772727,3300.0,5305.0,...,1955.0,1545.0,1170.0,850.0,610.0,1745.0,1260.0,485.0,27180.0,"Grandview-Woodland ,Vancouver,BC,Canada"
Hastings-Sunrise,15050.0,4.883721,15.415282,2.358804,75.913621,0.830565,37.674419,18.504983,4575.0,6315.0,...,1895.0,1465.0,1155.0,730.0,620.0,1505.0,1065.0,440.0,29920.0,"Hastings-Sunrise ,Vancouver,BC,Canada"
Kensington-Cedar Cottage,22135.0,3.54642,10.61667,2.100745,82.042015,1.106844,31.420827,31.488593,6595.0,8580.0,...,2915.0,1965.0,1490.0,1245.0,840.0,2245.0,1620.0,620.0,42685.0,"Kensington-Cedar Cottage ,Vancouver,BC,Canada"
Kerrisdale,6560.0,1.295732,11.204268,2.972561,82.469512,1.448171,41.768293,4.192073,2285.0,1650.0,...,680.0,670.0,460.0,370.0,315.0,2050.0,920.0,1130.0,13485.0,"Kerrisdale ,Vancouver,BC,Canada"
Killarney,14900.0,4.060403,8.758389,1.409396,82.818792,1.14094,27.751678,18.691275,4160.0,4700.0,...,1735.0,1320.0,950.0,910.0,520.0,1320.0,980.0,345.0,24900.0,"Killarney ,Vancouver,BC,Canada"
Kitsilano,11160.0,6.317204,43.996416,10.528674,35.080645,2.553763,8.915771,3.091398,4960.0,4945.0,...,3110.0,2410.0,2160.0,1805.0,1355.0,5940.0,3490.0,2450.0,43445.0,"Kitsilano ,Vancouver,BC,Canada"


In [80]:
'''
 Geopy provides a class for popular mapping services. Nominatim is the service behind the popular OpenStreetMap that allows you to geocode for free. But you should comply with the usage policies in order to allow everyone to use it:
'''
#from geopy import RateLimiter
locator = Nominatim(user_agent="http")

# 1 - convenient function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)


In [81]:
# 2- - create location column
df_new['location'] = df_new['Neighbourhood'].apply(geocode)

In [82]:
# 3 - create longitude, latitude and altitude from location column (returns tuple)
df_new['point'] = df_new['location'].apply(lambda loc: tuple(loc.point) if loc else None)

In [83]:
# 4 - split point column into latitude, longitude and altitude columns
df_new[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df_new['point'].tolist(), index=df_new.index)
df_new

,Total Population,Percentage of Latin-Americans,Percentage of Europeans,Percentage of US-Americans,Percentage of Asians,Percentage of Japanese,Percentage of Chinese,Percentage of South-East-Asians,Under $10.000 (including loss),$10.000 to $19.999,...,$100.000 and over,$100.000 to $149.999,$150.000 and over,Total,Neighbourhood,location,point,latitude,longitude,altitude
Arbutus-Ridge,7650.0,1.699346,14.117647,3.137255,78.69281,1.24183,43.464052,1.764706,2790.0,1820.0,...,1720.0,855.0,870.0,13915.0,"Arbutus-Ridge ,Vancouver,BC,Canada","(Arbutus-Ridge, Vancouver, Metro Vancouver Reg...","(49.2462132, -123.1612616, 0.0)",49.246213,-123.161262,0.0
Downtown,23240.0,5.658348,24.010327,4.668675,61.574871,2.560241,14.350258,4.410499,8785.0,8050.0,...,8590.0,5180.0,3405.0,62215.0,"Downtown ,Vancouver,BC,Canada","(Downtown, Vancouver, Metro Vancouver Regional...","(49.283393, -123.1174563, 0.0)",49.283393,-123.117456,0.0
Dunbar-Southlands,7810.0,3.008963,20.99872,6.850192,66.453265,1.024328,37.900128,2.432778,3320.0,2140.0,...,3515.0,1555.0,1965.0,20205.0,"Dunbar-Southlands ,Vancouver,BC,Canada","(Dunbar-Southlands, Vancouver, Metro Vancouver...","(49.237068550000004, -123.18436532910083, 0.0)",49.237069,-123.184365,0.0
Fairview,9295.0,6.777838,35.233997,8.552985,43.410436,2.582033,10.220549,5.325444,3185.0,3495.0,...,4105.0,2590.0,1520.0,33760.0,"Fairview ,Vancouver,BC,Canada","(Fairview, Vancouver, Metro Vancouver Regional...","(49.2619557, -123.13035055885713, 0.0)",49.261956,-123.130351,0.0
Grandview-Woodland,7480.0,6.684492,29.745989,7.219251,53.208556,1.938503,22.526738,14.772727,3300.0,5305.0,...,1745.0,1260.0,485.0,27180.0,"Grandview-Woodland ,Vancouver,BC,Canada","(Grandview-Woodland, Vancouver, Metro Vancouve...","(49.277138750000006, -123.06690083513674, 0.0)",49.277139,-123.066901,0.0
Hastings-Sunrise,15050.0,4.883721,15.415282,2.358804,75.913621,0.830565,37.674419,18.504983,4575.0,6315.0,...,1505.0,1065.0,440.0,29920.0,"Hastings-Sunrise ,Vancouver,BC,Canada","(Hastings-Sunrise, Vancouver, Metro Vancouver ...","(49.278752, -123.04000581430353, 0.0)",49.278752,-123.040006,0.0
Kensington-Cedar Cottage,22135.0,3.54642,10.61667,2.100745,82.042015,1.106844,31.420827,31.488593,6595.0,8580.0,...,2245.0,1620.0,620.0,42685.0,"Kensington-Cedar Cottage ,Vancouver,BC,Canada","(Kensington-Cedar Cottage, Vancouver, Metro Va...","(49.2476321, -123.0842067, 0.0)",49.247632,-123.084207,0.0
Kerrisdale,6560.0,1.295732,11.204268,2.972561,82.469512,1.448171,41.768293,4.192073,2285.0,1650.0,...,2050.0,920.0,1130.0,13485.0,"Kerrisdale ,Vancouver,BC,Canada","(Kerrisdale, Vancouver, Metro Vancouver Region...","(49.2209848, -123.1595483795563, 0.0)",49.220985,-123.159548,0.0
Killarney,14900.0,4.060403,8.758389,1.409396,82.818792,1.14094,27.751678,18.691275,4160.0,4700.0,...,1320.0,980.0,345.0,24900.0,"Killarney ,Vancouver,BC,Canada","(Killarney, Vancouver, Metro Vancouver Regiona...","(49.2242738, -123.0462504, 0.0)",49.224274,-123.046250,0.0
Kitsilano,11160.0,6.317204,43.996416,10.528674,35.080645,2.553763,8.915771,3.091398,4960.0,4945.0,...,5940.0,3490.0,2450.0,43445.0,"Kitsilano ,Vancouver,BC,Canada","(Kitsilano, Vancouver, Metro Vancouver Regiona...","(49.2694099, -123.155267, 0.0)",49.269410,-123.155267,0.0


In [84]:
# now apply geocoding on neighborhood
# get location of Toronto using geopy
from geopy.geocoders import Nominatim
address = 'Vancouver'
location = locator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print("The geographical coordinates of Vancouver are {}, {}".format(latitude, longitude))

The geographical coordinates of Vancouver are 49.2608724, -123.113952


In [85]:
# Folium mapping
import folium
map_Vancouver = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighbourhood in zip(df_new['latitude'], df_new['longitude'], df_new['Neighbourhood']):
  label = '{}'.format(neighbourhood)
  label = folium.Popup(label)
  folium.CircleMarker(
      [lat,lng],
      radius=8,
      color='blue',
      popup=label,
      fill_color='#3186cc',
      fill_opacity=0.7,
      fill=True

  ).add_to(map_Vancouver)

map_Vancouver

In [86]:
# load restaurants of Vancouver dataset
rest_path = pd.read_csv('Vancouver_restaurant_data.csv', encoding="latin")
df_rest = pd.DataFrame(rest_path)
df_rest



,Name of Restaurent,Type of Cusine,Rating,Total No of Ratings,Cost,Landmark,Opening time,Current Status,Dine in availability,Takeaway type,Delivery availability,Location and Address,Description of the Restaurent,Restaurent Name,Rating.1,number-of-reviews,veg /Non- Veg
0,Las Tortas Mexican,Restaurant,4.5,287,NaN,3353 Cambie Street,11:30AM,Closed,Dine-in,Takeaway,Delivery,NaN,NaN,NaN,NaN,NaN,Non-Veg
1,The Story Cafe -,Restaurant,4.7,154,NaN,2800 Sexsmith Road,9AM,Closed,Dine-in,Takeaway,Delivery,NaN,NaN,NaN,NaN,NaN,Non-Veg
2,Miku Vancouver,Sushi,4.6,"4,201",Expensive,NaN,11:30AM,Closed,NaN,NaN,NaN,200 Granville St # 70 ú In the Granville Square,Sushi & sustainable seafood are served in a so...,NaN,NaN,NaN,Non-Veg
3,Blue Water Cafe,Seafood,4.6,"1,822",Too Expensive,NaN,4:30PM,Closed,Dine-in,Takeaway,Delivery,1095 Hamilton St ú In Urban Systems,Destination in a heritage warehouse space offe...,Blue Water Cafe,4.6,(1822),Non-Veg
4,Chambar Restaurant,Restaurant,4.4,"2,736",Expensive,NaN,11:30AM,Closed,Dine-in,Takeaway,Delivery,568 Beatty St,Chic exposed-brick space serving creative Belg...,Chambar Restaurant,4.4,(2736),Non-Veg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,Shangri-La Hotel Vancouver Drive way,No reviews Hotel,NaN,NaN,NaN,"Vancouver, BC, Canada",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Shangri-La Hotel Vancouver Drive way,NaN,NaN,Non-Veg
380,Hilton Vancouver Metrotown,Hotel,4.2,"1,518",NaN,"Burnaby, BC, Canada",NaN,NaN,NaN,NaN,NaN,NaN,Modern hotel with a heated rooftop pool & hot ...,Hilton Vancouver Metrotown,4.2,(1518),Non-Veg
381,Vancouver International Airport,Airport,4.2,"4,148",NaN,"Richmond, BC, Canada",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vancouver International Airport,4.2,(4148),Non-Veg
382,Radisson Hotel Vancouver Airport,Hotel,4.0,841,NaN,"Richmond, BC, Canada",NaN,NaN,NaN,NaN,NaN,NaN,Polished hotel with free Wi-Fi & an airport sh...,Radisson Hotel Vancouver Airport,4.0,(841),Non-Veg


In [87]:
# drop unuseful features
df_rest.drop(['Rating.1', 'number-of-reviews','veg /Non- Veg','Rating','Total No of Ratings','Current Status','Opening time','Dine in availability','Takeaway type','Delivery availability','Description of the Restaurent','Restaurent Name','Cost'], axis=1, inplace=True)
df_rest.head()


,Name of Restaurent,Type of Cusine,Landmark,Location and Address
0,Las Tortas Mexican,Restaurant,3353 Cambie Street,NaN
1,The Story Cafe -,Restaurant,2800 Sexsmith Road,NaN
2,Miku Vancouver,Sushi,NaN,200 Granville St # 70 ú In the Granville Square
3,Blue Water Cafe,Seafood,NaN,1095 Hamilton St ú In Urban Systems
4,Chambar Restaurant,Restaurant,NaN,568 Beatty St


In [88]:
# first: replace nan with ''
values = {"Landmark": '', "Location and Address": ''}
df_rest = df_rest.fillna(value = values)

# merge place and landmark, since they have the same meaning
df_rest['Place'] = df_rest['Landmark'] + df_rest['Location and Address']
df_rest.drop(['Landmark'], axis=1, inplace=True)
df_rest.drop(['Location and Address'], axis=1, inplace=True)
print(df_rest.size)

1152


In [89]:
df_rest.Place.value_counts()

Vancouver, BC, Canada                                     209
Richmond, BC, Canada                                        8
North Vancouver, BC, Canada                                 8
West Vancouver, BC, Canada                                  5
Vancouver, BC, Canada ú Near the Vancouver Art Gallery      5
                                                         ... 
Vancouver, BC, Canada ú In The Dominion Building            1
Vancouver, BC, Canada ú In the Fairmont Pacific Rim         1
Vancouver, BC, Canada ú In Lee Bldg                         1
Vancouver, BC, Canada ú In Burrard Landing                  1
Victoria, BC, Canada                                        1
Name: Place, Length: 148, dtype: int64

In [90]:
df_rest.value_counts(df_rest["Type of Cusine"])

Type of Cusine
Restaurant                    74
Indian                        27
Italian                       22
Seafood                       14
Japanese                      13
                              ..
Latin American                 1
Lebanese                       1
No reviews Business broker     1
No reviews Hotel               1
Yakiniku                       1
Length: 95, dtype: int64

In [91]:
# problem: Type of cuisine "Restaurant" doesn't have a meaning
# -> remove it
cnt = 0
for element in range(len(df_rest["Type of Cusine"])):
    rest_name = ""
    if df_rest.loc[element,"Type of Cusine"] == "Restaurant":
        print(df_rest.loc[[element]])
        rest_name = df_rest.loc[element,"Name of Restaurent "]
        if (re.search("Mexican",rest_name)) or (re.search("Taco",rest_name)) or re.search("Brazil",rest_name):
            df_rest.loc[element,"Type of Cusine"] = "Latin"
            print(df_rest.loc[[element]])
        elif (re.search("French",rest_name)) or re.search("Italian",rest_name) or re.search("Steak",rest_name) or re.search("Provence",rest_name) or re.search("Mangez",rest_name) or re.search("Mediterran",rest_name) or re.search("Pizza",rest_name):
            df_rest.loc[element,"Type of Cusine"] = "Western"
        elif (re.search("Cafe",rest_name)):
            df_rest.loc[element,"Type of Cusine"] = "Cafe"
        elif (re.search("Asia",rest_name)) or re.search("Thai",rest_name):
            df_rest.loc[element,"Type of Cusine"] = "Asian"
        elif re.search("Ramen",rest_name):
            df_rest.loc[element,"Type of Cusine"] = "Japanese"
        elif re.search("Bar",rest_name):
            df_rest.loc[element,"Type of Cusine"] = "Bar"
        else:
            df_rest = df_rest.drop(df_rest.index[[element-cnt]])
            cnt += 1
df_rest.reset_index(drop=True, inplace=True)

  Name of Restaurent  Type of Cusine               Place
0  Las Tortas Mexican     Restaurant  3353 Cambie Street
  Name of Restaurent  Type of Cusine               Place
0  Las Tortas Mexican          Latin  3353 Cambie Street
  Name of Restaurent  Type of Cusine               Place
1    The Story Cafe -     Restaurant  2800 Sexsmith Road
  Name of Restaurent  Type of Cusine          Place
4  Chambar Restaurant     Restaurant  568 Beatty St
     Name of Restaurent  Type of Cusine  \
5  Hawksworth Restaurant     Restaurant   

                                              Place  
5  801 W Georgia St ú In the Rosewood Hotel Georgia  
   Name of Restaurent  Type of Cusine                  Place
33       Brix & Mortar     Restaurant  Vancouver, BC, Canada
      Name of Restaurent  Type of Cusine                  Place
45  Heritage Asian Eatery     Restaurant  Vancouver, BC, Canada
   Name of Restaurent  Type of Cusine                  Place
47       The Red Wagon     Restaurant  Vancouver

In [92]:
 # there are duplicates in the dataframe -> remove them
df_rest[df_rest.duplicated(keep = False)]

,Name of Restaurent,Type of Cusine,Place
126,Earls Kitchen + Bar,Bar,"Vancouver, BC, Canada"
164,Earls Kitchen + Bar,Bar,"Vancouver, BC, Canada"
221,Pokerrito,Hawaiian,"Vancouver, BC, Canada"
230,Pokerrito,Hawaiian,"Vancouver, BC, Canada"
255,A&W Canada,Hamburger,"Vancouver, BC, Canada"
256,A&W Canada,Hamburger,"Vancouver, BC, Canada"


In [93]:
df_rest = df_rest.drop(df_rest.index[256])
df_rest = df_rest.drop(df_rest.index[230])
df_rest = df_rest.drop(df_rest.index[164])
df_rest[df_rest.duplicated(keep = False)] # no duplicates anymore
df_rest.reset_index(drop=True, inplace=True)

In [94]:
df_rest

,Name of Restaurent,Type of Cusine,Place
0,Las Tortas Mexican,Latin,3353 Cambie Street
1,The Story Cafe -,Cafe,2800 Sexsmith Road
2,Miku Vancouver,Sushi,200 Granville St # 70 ú In the Granville Square
3,Blue Water Cafe,Seafood,1095 Hamilton St ú In Urban Systems
4,Forage,Pacific Northwest,1300 Robson St
...,...,...,...
331,Shangri-La Hotel Vancouver Drive way,No reviews Hotel,"Vancouver, BC, Canada"
332,Hilton Vancouver Metrotown,Hotel,"Burnaby, BC, Canada"
333,Vancouver International Airport,Airport,"Richmond, BC, Canada"
334,Radisson Hotel Vancouver Airport,Hotel,"Richmond, BC, Canada"


In [95]:
df_rest.value_counts(df_rest["Type of Cusine"])

Type of Cusine
Indian                        27
Italian                       22
Seafood                       14
Japanese                      14
Bar                           11
                              ..
Latin American                 1
Lebanese                       1
No reviews Business broker     1
No reviews Hotel               1
Yakiniku                       1
Length: 97, dtype: int64

In [96]:
# put type of cuisines together
cnt = 0
for element in range(len(df_rest["Type of Cusine"])):
    type = df_rest.loc[element,"Type of Cusine"]
    if (re.search("French",type)) or re.search("Italian",type) or re.search("Vegetarian",type) or re.search("European",type) or re.search("Steak",type) or re.search("Mediterran",type) or re.search("Pizza",type)  or re.search("Hawaii",type)  or re.search("Americ",type) or re.search("Canad",type)  or re.search("Salad",type)  or re.search("Spanish",type)  or re.search("Tapas",type)   or re.search("Pacific Northwest",type) or re.search("German",type) or re.search("Irish",type) or re.search("cois",type) or re.search("Tuscan",type) or re.search("Continental",type) or re.search("Hamburger",type) or re.search("Fast",type) or re.search("Cantonese",type) or re.search("Breakfast",type) or re.search("Sandwich",type):
        df_rest.loc[element,"Type of Cusine"] = "Western"
        print(df_rest.loc[[element]])
    elif (re.search("Asia",type)) or re.search("Cambodia",type) or re.search("Pho",type) or re.search("Fusion",type) or re.search("Thai",type) or re.search("Vietnamese",type) or re.search("Lebanese",type) or re.search("Afgha",type) or re.search("Ethiop",type) or re.search("Persia",type):
        df_rest.loc[element,"Type of Cusine"] = "Asian"
        print(df_rest.loc[[element]])
    elif re.search("Ramen",type) or re.search("Sushi",type) or re.search("Terri",type) or re.search("Izakaya",type) or re.search("Yakin",type) or re.search("Japanese",type) or re.search("Teppan",type)  or re.search("Seafood",type):
        df_rest.loc[element,"Type of Cusine"] = "Japanese"
        print(df_rest.loc[[element]])
    elif re.search("Chines",type):
        df_rest.loc[element,"Type of Cusine"] = "Chinese"
        print(df_rest.loc[[element]])
    else:
        df_rest = df_rest.drop(df_rest.index[[element-cnt]])
        cnt += 1
df_rest.reset_index(drop=True, inplace=True)

  Name of Restaurent  Type of Cusine  \
2      Miku Vancouver       Japanese   

                                             Place  
2  200 Granville St # 70 ú In the Granville Square  
  Name of Restaurent  Type of Cusine                                Place
3     Blue Water Cafe       Japanese  1095 Hamilton St ú In Urban Systems
  Name of Restaurent  Type of Cusine           Place
4              Forage        Western  1300 Robson St
      Name of Restaurent  Type of Cusine          Place
5  St Lawrence Restaurant        Western  269 Powell St
  Name of Restaurent  Type of Cusine  \
6            Botanist        Western   

                                          Place  
6  1038 Canada Pl ú In the Fairmont Pacific Rim  
              Name of Restaurent  Type of Cusine  \
7  Hy's Steakhouse & Cocktail Bar        Western   

                                            Place  
7  637 Hornby St ú Near the Vancouver Art Gallery  
  Name of Restaurent  Type of Cusine          Place
8    

In [97]:
df_rest.value_counts(df_rest["Type of Cusine"])

Type of Cusine
Western     96
Japanese    48
Asian       21
Chinese     11
dtype: int64

In [98]:
df_rest

,Name of Restaurent,Type of Cusine,Place
0,Miku Vancouver,Japanese,200 Granville St # 70 ú In the Granville Square
1,Blue Water Cafe,Japanese,1095 Hamilton St ú In Urban Systems
2,Forage,Western,1300 Robson St
3,St Lawrence Restaurant,Western,269 Powell St
4,Botanist,Western,1038 Canada Pl ú In the Fairmont Pacific Rim
...,...,...,...
171,Umi Sushi Express,Japanese,"Vancouver, BC, Canada"
172,Kirin Seafood Restaurant,Chinese,"Richmond, BC, Canada"
173,Haida Sandwich,Western,"North Vancouver, BC, Canada"
174,Golden Paramount Seafood Restaurant,Japanese,"Richmond, BC, Canada"


In [99]:
for row in range(len(df_rest)):
    rest = df_rest.loc[row,'Name of Restaurent ']
    address = df_rest.loc[row,'Place']
    if address == "Vancouver, BC, Canada":
        df_rest.loc[row,'Place'] = rest + ",Vancouver, BC, Canada"

In [100]:
df_rest.value_counts(df_rest["Place"])

Place
West Vancouver, BC, Canada                                4
Richmond, BC, Canada                                      3
North Vancouver, BC, Canada                               2
Vancouver, BC, Canada ú Near the Vancouver Art Gallery    2
                                                          1
                                                         ..
755 Burrard St #219                                       1
780 Thurlow St                                            1
845 Burrard St ú Near the Vancouver Art Gallery           1
860 Burrard St ú Near the Vancouver Art Gallery           1
vancouver, BC, Canada                                     1
Length: 169, dtype: int64

In [101]:
# put addresses in a proper way so that we can feed it into geopy
for row in range(len(df_rest)):
    address = df_rest.loc[row,'Place']
    address_lst = []
    if address.find("ú In") != -1:
        address_lst = address.split("ú In")
        if address_lst[0] == "Vancouver,BC,Canada":
            df_rest.loc[row,"Place"] = address_lst[1] + ", Vancouver,BC, CA"
        else:
            df_rest.loc[row,"Place"] = address_lst[0] + ",Vancouver, BC, Canada"
    elif address.find("ú Near") != -1:
        address_lst = address.split("ú Near")
        df_rest.loc[row,"Place"] = address_lst[0] + ",Vancouver, BC, Canada"
    elif address.find("#") != -1:
        address_lst = address.split("#")
        df_rest.loc[row,"Place"] = address_lst[0] + ",Vancouver, BC, Canada"
    elif address.find("ancouver") != -1:
        df_rest.loc[row,"Place"] = "Vancouver, BC, Canada"
    elif address.find("BC, Canada") == -1 :
        df_rest.loc[row,"Place"] = address + ",Vancouver, BC, Canada"
df_rest.reset_index(drop=True, inplace=True)


In [102]:
df_rest.value_counts(df_rest["Place"])

Place
Vancouver, BC, Canada                           89
Vancouver, BC, Canada ,Vancouver, BC, Canada    11
Richmond, BC, Canada                             3
755 Burrard St ,Vancouver, BC, Canada            2
New Westminster, BC, Canada                      1
                                                ..
1809 W 1st Ave,Vancouver, BC, Canada             1
1696 Duranleau St,Vancouver, BC, Canada          1
163 Keefer St,Vancouver, BC, Canada              1
1600 Howe St ,Vancouver, BC, Canada              1
370 Cambie St,Vancouver, BC, Canada              1
Length: 75, dtype: int64

In [103]:
for row in range(len(df_rest)):
    address = df_rest.loc[row,'Place']
    if address.find("#") != -1:
        address_a = re.sub(r"# \d+", '', address)
        df_rest.loc[row,"Place"] = address_a
    elif address.find("Suite") != -1:
        address_a = re.sub(r"Suite \d+", '', address)
        df_rest.loc[row,"Place"] = address_a
    elif address.find("St") != -1:
        address_a = re.sub(r"g\d+", '', address)
        df_rest.loc[row,"Place"] = address_a
    elif address.find("Vancouver, BC, Canada ,Vancouver, BC, Canada") != -1:
        df_rest.loc[row,"Place"] = "Vancouver, BC, Canada"

In [104]:
for row in range(len(df_rest)):
    rest = df_rest.loc[row,'Name of Restaurent ']
    address = df_rest.loc[row,'Place']
    if address == "Vancouver, BC, Canada":
        df_rest.loc[row,'Place'] = rest + ",Vancouver, BC, Canada"

In [105]:
df_rest.value_counts(df_rest["Place"])

Place
Richmond, BC, Canada                             3
755 Burrard St ,Vancouver, BC, Canada            2
,Vancouver, BC, Canada                           1
O Sushi,Vancouver, BC, Canada                    1
Kingyo,Vancouver, BC, Canada                     1
                                                ..
845 Burrard St ,Vancouver, BC, Canada            1
860 Burrard St ,Vancouver, BC, Canada            1
888 Nelson St ,Vancouver, BC, Canada             1
891 Homer St,Vancouver, BC, Canada               1
Zen Japanese Restaurant,Vancouver, BC, Canada    1
Length: 173, dtype: int64

In [106]:
df_rest

,Name of Restaurent,Type of Cusine,Place
0,Miku Vancouver,Japanese,"200 Granville St ,Vancouver, BC, Canada"
1,Blue Water Cafe,Japanese,"1095 Hamilton St ,Vancouver, BC, Canada"
2,Forage,Western,"1300 Robson St,Vancouver, BC, Canada"
3,St Lawrence Restaurant,Western,"269 Powell St,Vancouver, BC, Canada"
4,Botanist,Western,"1038 Canada Pl ,Vancouver, BC, Canada"
...,...,...,...
171,Umi Sushi Express,Japanese,"Umi Sushi Express,Vancouver, BC, Canada"
172,Kirin Seafood Restaurant,Chinese,"Richmond, BC, Canada"
173,Haida Sandwich,Western,"Haida Sandwich,Vancouver, BC, Canada"
174,Golden Paramount Seafood Restaurant,Japanese,"Richmond, BC, Canada"


In [107]:
# 1 - convenient function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)

In [108]:
# 2- - create location column
df_rest['location'] = df_rest['Place'].apply(geocode)

In [109]:
for row in range(len(df_rest['location'])):
    element = df_rest.loc[row, 'location']
    if element == None:
        df_rest = df_rest.drop([row])
df_rest.reset_index(drop=True, inplace=True)

In [110]:
# 3 - create longitude, latitude and altitude from location column (returns tuple)
df_rest['point'] = df_rest['location'].apply(lambda loc: tuple(loc.point) if loc else None)

In [111]:
# 4 - split point column into latitude, longitude and altitude columns
df_rest[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df_rest['point'].tolist(), index=df_rest.index)

In [112]:
df_rest

,Name of Restaurent,Type of Cusine,Place,location,point,latitude,longitude,altitude
0,Miku Vancouver,Japanese,"200 Granville St ,Vancouver, BC, Canada","(The Vancouver Sun, 200, Granville Street, Gas...","(49.2867868, -123.11233418001919, 0.0)",49.286787,-123.112334,0.0
1,Blue Water Cafe,Japanese,"1095 Hamilton St ,Vancouver, BC, Canada","(Hamilton Street, Chinatown, Downtown, Vancouv...","(49.2807695, -123.1129959, 0.0)",49.280769,-123.112996,0.0
2,Forage,Western,"1300 Robson St,Vancouver, BC, Canada","(The Listel Hotel, 1300, Robson Street, Davie ...","(49.2872256, -123.1291922, 0.0)",49.287226,-123.129192,0.0
3,St Lawrence Restaurant,Western,"269 Powell St,Vancouver, BC, Canada","(St Lawrence Restaurant, 269, Powell Street, D...","(49.2832655, -123.0978958, 0.0)",49.283265,-123.097896,0.0
4,Botanist,Western,"1038 Canada Pl ,Vancouver, BC, Canada","(Fairmont Pacific Rim, 1038, Canada Place, Gas...","(49.2880994, -123.116883, 0.0)",49.288099,-123.116883,0.0
...,...,...,...,...,...,...,...,...
133,Hula,Western,"Hula,Vancouver, BC, Canada","(Hula, 1076, Mainland Street, Yaletown, Downto...","(49.2755974, -123.120309, 0.0)",49.275597,-123.120309,0.0
134,KINTON RAMEN,Japanese,"KINTON RAMEN,Vancouver, BC, Canada","(KINTON RAMEN, 6111, University Boulevard, Uni...","(49.2661144, -123.2483285, 0.0)",49.266114,-123.248328,0.0
135,Kirin Seafood Restaurant,Chinese,"Richmond, BC, Canada","(Richmond, Metro Vancouver Regional District, ...","(49.163168, -123.137414, 0.0)",49.163168,-123.137414,0.0
136,Golden Paramount Seafood Restaurant,Japanese,"Richmond, BC, Canada","(Richmond, Metro Vancouver Regional District, ...","(49.163168, -123.137414, 0.0)",49.163168,-123.137414,0.0


In [113]:
import folium
map_Vancouver_rest = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighbourhood in zip(df_rest['latitude'], df_rest['longitude'], df_rest['Place']):
  label = '{}'.format(neighbourhood)
  label = folium.Popup(label)
  folium.CircleMarker(
      [lat,lng],
      radius=8,
      color='blue',
      popup=label,
      fill_color='#3186cc',
      fill_opacity=0.7,
      fill=True

  ).add_to(map_Vancouver_rest)

map_Vancouver_rest